CISC442 PR2
Jenn Werth

Note: change the Runtime to T4 GPU

In [2]:
# Import required modules and libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [3]:
# Resize the size of all images to a unanimous value (224, 224).
# Convert PIL image objects into Tensors.
# Normalize the tensor values based on the mean and standard deviation of the RGB values of all the images

data_transforms = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(
      mean = [0.485, 0.456, 0.406],   # Usimg mean and std dev of ImageNet
      std = [0.229, 0.224, 0.225])
])

In [4]:
# Create an object of torchvision.datasets.CIFAR100 to get the training and testing set:

# setting the root to the current working directory
trainset = datasets.CIFAR100('.', train = True, transform = data_transforms,
download=True)

testset = datasets.CIFAR100('.', train=False, transform = data_transforms,
download=True)

100%|██████████| 169001437/169001437 [00:12<00:00, 13208148.14it/s]


Extracting ./cifar-100-python.tar.gz to .
Files already downloaded and verified


In [5]:
# Create a data loader.

# Making one for the training and one for test set
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

# Batch sized used with Pytorch documentation = 64

In [6]:
# Load a VGG16 network with pretrained ImageNet weights:
model = models.vgg16(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 153MB/s]


In [7]:
# Extract the number of input features for the last fully connected layer of the model:
num_in_ftrs = model.classifier[6].in_features
print(num_in_ftrs)

4096


In [8]:
# Replace the last fully connected layer with a new layer.
# The new layer has the same number of input features as the original network
# but the number of outputs should be equal to the number of classes in the CIFAR100 dataset.

num_cls = 100 # number of classes in the CIFAR100 dataset
model.classifier[6] = nn.Linear(num_in_ftrs, num_cls)

In [9]:
# We are using pretrained weights from the ImageNet dataset. The last layer of VGG16 has been replaced
# for fitting with our dataset (CIFAR100).
# Except for the new last layer, weights from other layers need to be frozen.
# It means that these weights will not be updated during the training.

for param in model.parameters(): # freeze all the layers
  param.requires_grad = False
for param in model.classifier[6].parameters(): # unfreeze the last linear layer.
  param.requires_grad = True

In [10]:
# Set the number of epochs:
num_epochs = 10

In [11]:
# Move the model to GPU (if available):
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [12]:
# Define a loss function for evaluating the trained model:
criterion = nn.CrossEntropyLoss()

In [13]:
# Create an optimizer with an initial learning rate and momentum:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
# Create a scheduler to control the way that learning rate changes during the training process:
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [15]:
# Iterate over the epochs and save the best model weights.

model.train() # set to training mode

# extract the images and labels from each epoch
for i in range(num_epochs):
  for _, data in enumerate(train_loader, 0):
    images, labels = data
    images, labels = images.to(device), labels.to(device) # move to cpu from gpu
    optimizer.zero_grad()
    outputs = model(images) # here the model is the pretrained VGG16
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  scheduler.step()

# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

# Save the best model
torch.save(model.state_dict(), 'best_model.pth')


In [17]:
# Testing process
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [19]:
# Set model to evaluation / testing mode
model.eval()

# define vars for accuracy evaluation later
correct = 0
total = 0

with torch.no_grad():
  for i, data in enumerate(test_loader, 0):
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)

accuracy = (correct / total) * 100
print(accuracy)


61.14000000000001
